# 🚀 LLM Fine-Tuning with QLoRA — Interactive Training Notebook

This notebook walks you through **every step** of fine-tuning a Large Language Model (Mistral-7B) on a **text-to-SQL** task using **QLoRA** (Quantised Low-Rank Adaptation).

## What You'll Learn

| Step | Concept | Why It Matters |
|------|---------|----------------|
| 1 | Environment Setup | Install the right versions of everything |
| 2 | Dataset Loading | Use HF `datasets` to load text-to-SQL data |
| 3 | Prompt Engineering | Format data as instruction-style prompts |
| 4 | Quantisation (BitsAndBytes) | Compress 7B model from 14 GB → 4.5 GB |
| 5 | LoRA Adapter Setup | Train only 0.4% of parameters |
| 6 | Training with SFTTrainer | Supervised fine-tuning loop |
| 7 | Inference | Generate SQL from natural language |
| 8 | Evaluation | Measure perplexity, exact match, BLEU |

---

**Prerequisites:**
- A CUDA-capable GPU (16+ GB VRAM recommended, e.g. T4, A10, L4, A100)
- Python 3.9+
- Hugging Face account (for model access)

## Step 1: Environment Setup

Install all dependencies. If running on Google Colab, the cell below handles everything.

In [ ]:
# ============================================================================
# STEP 1: Install Dependencies
# ============================================================================
# We install pinned versions to ensure reproducibility.
# On Colab, many of these are pre-installed but may need upgrading.

!pip install -q \
    transformers>=4.36.0 \
    datasets>=2.16.0 \
    accelerate>=0.25.0 \
    peft>=0.7.0 \
    trl>=0.7.0 \
    bitsandbytes>=0.41.0 \
    wandb>=0.16.0 \
    evaluate>=0.4.0 \
    rouge-score \
    sqlparse \
    rich \
    pyyaml

# Verify GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available:  {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU:             {torch.cuda.get_device_name(0)}")
    print(f"VRAM:            {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")

In [ ]:
# ============================================================================
# STEP 1b: (Optional) Login to Hugging Face & Weights and Biases
# ============================================================================
# Hugging Face login is needed to download gated models (e.g. Llama-2).
# Mistral-7B is NOT gated, so you can skip the HF login.

# Uncomment if you need HF login:
# from huggingface_hub import login
# login()

# Uncomment for W&B tracking:
# import wandb
# wandb.login()

## Step 2: Load the Dataset

We use the **b-mc2/sql-create-context** dataset — a curated collection of natural-language questions paired with SQL schemas and gold SQL queries.

Each example has:
- `question`: "How many employees are in department 5?"
- `context`: `CREATE TABLE employees (id INT, name TEXT, dept_id INT)`
- `answer`: `SELECT COUNT(*) FROM employees WHERE dept_id = 5`

In [ ]:
# ============================================================================
# STEP 2: Load Dataset from Hugging Face Hub
# ============================================================================
from datasets import load_dataset

# load_dataset() downloads from https://huggingface.co/datasets/b-mc2/sql-create-context
# It caches the data locally (~50 MB) so re-runs are instant.
dataset = load_dataset("b-mc2/sql-create-context", split="train")

print(f"Total examples: {len(dataset):,}")
print(f"Columns:        {dataset.column_names}")
print(f"\nSample example:")
print(f"  Question: {dataset[0]['question']}")
print(f"  Schema:   {dataset[0]['context'][:100]}...")
print(f"  Answer:   {dataset[0]['answer']}")

In [ ]:
# ============================================================================
# STEP 2b: Subsample for Quick Experimentation
# ============================================================================
# Training on the full dataset takes hours on a single GPU.
# Start with 10% to validate your pipeline, then scale up.

SUBSET_FRACTION = 0.1  # Use 10% of the data  (change to 1.0 for full run)

num_examples = int(len(dataset) * SUBSET_FRACTION)
dataset = dataset.shuffle(seed=42).select(range(num_examples))
print(f"Using {SUBSET_FRACTION:.0%} subset: {len(dataset):,} examples")

## Step 3: Prompt Engineering

LLMs learn from **structured prompts**. We convert each raw example into an instruction format that teaches the model:

```
Below is a question about a database along with the schema.
Write the SQL query that answers the question.

### Question:
{the actual question}

### Schema:
{CREATE TABLE statements}

### SQL:
{the gold answer}
```

**Why this format?** The model learns the pattern: when it sees `### SQL:`, it should generate a SQL query based on the question and schema above.

In [ ]:
# ============================================================================
# STEP 3: Format Prompts
# ============================================================================

PROMPT_TEMPLATE = """Below is a question about a database along with the schema.
Write the SQL query that answers the question.

### Question:
{question}

### Schema:
{context}

### SQL:
{answer}"""


def format_example(example):
    """
    Convert a dataset row into an instruction-style prompt.
    
    The .format() method replaces {question}, {context}, {answer}
    with the actual values from the dataset row.
    """
    example["text"] = PROMPT_TEMPLATE.format(
        question=example["question"],
        context=example["context"],
        answer=example["answer"],
    )
    return example


# Apply to every example using .map()
# .map() is the standard way to transform datasets in Hugging Face.
# It's lazy, cached, and supports multiprocessing.
dataset = dataset.map(format_example)

# Let's see what one looks like
print("=" * 60)
print("FORMATTED PROMPT EXAMPLE:")
print("=" * 60)
print(dataset[0]["text"])

In [ ]:
# ============================================================================
# STEP 3b: Split into Train / Validation / Test
# ============================================================================
# The dataset comes as a single split, so we create our own:
#   - Train (80%): used for gradient updates
#   - Validation (10%): used for early stopping / hyper-param tuning
#   - Test (10%): used for final evaluation (never seen during training)

# First, carve out the test set
train_val_test = dataset.train_test_split(test_size=0.1, seed=42)

# Then split the remainder into train and validation
train_val = train_val_test["train"].train_test_split(test_size=0.111, seed=42)
# 0.111 of 0.9 ≈ 0.1 of the total → 10% validation

train_dataset = train_val["train"]
val_dataset = train_val["test"]
test_dataset = train_val_test["test"]

print(f"Train:      {len(train_dataset):,} examples")
print(f"Validation: {len(val_dataset):,} examples")
print(f"Test:       {len(test_dataset):,} examples")

## Step 4: Load the Model with QLoRA Quantisation

### What is QLoRA?

**QLoRA = Quantised LoRA** — it combines two techniques:

1. **Quantisation** (BitsAndBytes): Compress the 7B model from fp16 (14 GB) to 4-bit NF4 (~4.5 GB)
2. **LoRA**: Train small adapter matrices (~50 MB) on top of the frozen, quantised base

This means you can fine-tune a 7B model on a **single consumer GPU** (16 GB VRAM)!

### Key Quantisation Concepts

| Concept | What It Does |
|---------|-------------|
| **NF4** (Normal Float 4) | 4-bit data type optimised for normally-distributed weights |
| **Double Quantisation** | Quantises the quantisation constants too (saves ~0.4 GB) |
| **Compute dtype** | The precision used during forward/backward (fp16 for speed) |

In [ ]:
# ============================================================================
# STEP 4a: Configure 4-bit Quantisation
# ============================================================================
from transformers import BitsAndBytesConfig
import torch

# BitsAndBytesConfig tells from_pretrained() HOW to load the weights.
# Instead of loading 16-bit floats, it loads 4-bit NF4 integers.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                     # Enable 4-bit loading
    bnb_4bit_quant_type="nf4",             # NF4 is best for normally-distributed weights
    bnb_4bit_compute_dtype=torch.float16,   # Compute in fp16 during forward pass
    bnb_4bit_use_double_quant=True,         # Quantise the quantisation constants too
)

print("Quantisation config created:")
print(f"  - 4-bit NF4 quantisation")
print(f"  - Double quantisation: ON")
print(f"  - Compute dtype: float16")

In [ ]:
# ============================================================================
# STEP 4b: Load the Base Model (Quantised)
# ============================================================================
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "mistralai/Mistral-7B-v0.1"

# Load the tokeniser
# The tokeniser converts text → token IDs and back.
# It's separate from the model and is NOT quantised.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# IMPORTANT: Many models don't define a pad token.
# Without it, batched training crashes.  We use EOS as a safe fallback.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Pad on the right for causal LM

# Load the model with quantisation
# device_map="auto" places layers across available GPUs automatically.
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)

# Disable caching (incompatible with gradient checkpointing)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Check model size
total_params = sum(p.numel() for p in model.parameters())
print(f"\nModel loaded: {MODEL_NAME}")
print(f"Total parameters: {total_params / 1e9:.2f}B")
print(f"Model dtype: {next(model.parameters()).dtype}")

## Step 5: Attach LoRA Adapters

### How LoRA Works

Instead of updating all 7B parameters, LoRA inserts **small trainable matrices** into specific layers:

```
W' = W + (α/r) × B @ A

where:
  W  ∈ R^{d×k}  — original frozen weight (e.g. 4096×4096)
  A  ∈ R^{d×r}  — down-projection (trainable)
  B  ∈ R^{r×k}  — up-projection (trainable, initialised to 0)
  r              — rank (e.g. 16, much smaller than d)
  α              — scaling factor
```

**Parameter savings:**
- Original layer: 4096 × 4096 = 16.8M parameters
- LoRA adapter (r=16): 4096×16 + 16×4096 = 131K parameters
- **That's 0.78% of the original!**

In [ ]:
# ============================================================================
# STEP 5: Configure and Attach LoRA Adapters
# ============================================================================
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Step 5a: Prepare the quantised model for training
# This casts layer norms to float32 for numerical stability
# and enables gradient computation on input embeddings.
model = prepare_model_for_kbit_training(model)

# Step 5b: Define the LoRA configuration
lora_config = LoraConfig(
    r=16,                    # Rank: dimensionality of A and B matrices
    lora_alpha=32,           # Scaling factor (rule of thumb: 2 × r)
    lora_dropout=0.05,       # Dropout on LoRA layers (regularisation)
    target_modules=[         # Which layers get LoRA adapters
        "q_proj",            # Attention: query projection
        "k_proj",            # Attention: key projection
        "v_proj",            # Attention: value projection
        "o_proj",            # Attention: output projection
        "gate_proj",         # MLP: gate projection (SwiGLU)
        "up_proj",           # MLP: up projection
        "down_proj",         # MLP: down projection
    ],
    bias="none",             # Don't train bias terms
    task_type="CAUSAL_LM",   # Task type for causal language modelling
)

# Step 5c: Wrap the model — freezes base, makes LoRA trainable
model = get_peft_model(model, lora_config)

# Print the parameter summary
model.print_trainable_parameters()
# Expected output:
# trainable params: 13,631,488 || all params: 3,514,900,480 || trainable%: 0.39

## Step 6: Training

We use **SFTTrainer** from the `trl` library — a Trainer subclass designed for Supervised Fine-Tuning.

### Key Training Parameters

| Parameter | Value | Why |
|-----------|-------|-----|
| `learning_rate` | 2e-4 | Standard for QLoRA; higher than full fine-tuning |
| `batch_size × grad_accum` | 4 × 4 = 16 | Effective batch of 16 without needing 16× memory |
| `num_epochs` | 3 | 1–3 is typical for fine-tuning |
| `lr_scheduler` | cosine | Smooth decay to near-zero |
| `gradient_checkpointing` | ON | Saves ~30% VRAM by recomputing activations |

In [ ]:
# ============================================================================
# STEP 6: Configure Training & Launch
# ============================================================================
from transformers import TrainingArguments
from trl import SFTTrainer

# Training arguments control EVERYTHING about the training loop:
# batch size, learning rate, checkpointing, logging, etc.
training_args = TrainingArguments(
    output_dir="./results",                   # Where to save checkpoints
    num_train_epochs=3,                        # Total passes over the dataset
    per_device_train_batch_size=4,             # Batch size PER GPU
    per_device_eval_batch_size=4,              # Eval batch size
    gradient_accumulation_steps=4,             # Effective batch = 4 × 4 = 16
    learning_rate=2e-4,                        # Peak learning rate
    weight_decay=0.001,                        # L2 regularisation
    lr_scheduler_type="cosine",                # Cosine annealing schedule
    warmup_ratio=0.03,                         # Warm up for 3% of total steps
    fp16=True,                                 # Mixed precision (float16)
    gradient_checkpointing=True,               # Trade compute for memory
    logging_steps=25,                          # Log every 25 steps
    save_strategy="steps",                     # Save checkpoints by step count
    save_steps=200,                            # Save every 200 steps
    save_total_limit=3,                        # Keep only last 3 checkpoints
    eval_strategy="steps",                     # Evaluate by step count
    eval_steps=200,                            # Evaluate every 200 steps
    optim="paged_adamw_32bit",                 # QLoRA-friendly optimiser
    seed=42,                                   # Reproducibility
    group_by_length=True,                      # Group similar lengths (less padding)
    report_to="none",                          # Set to "wandb" to enable tracking
    remove_unused_columns=False,               # Keep all dataset columns
)

# SFTTrainer handles:
# - Causal LM loss computation (labels = shifted input_ids)
# - Sequence packing (multiple short examples in one sequence)
# - PEFT integration (only computes gradients for LoRA params)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=lora_config,
    tokenizer=tokenizer,
    args=training_args,
    max_seq_length=512,              # Max tokens per sequence
    dataset_text_field="text",       # Column with our formatted prompts
)

print("Trainer initialised. Ready to train!")
print(f"  Training examples:   {len(train_dataset):,}")
print(f"  Validation examples: {len(val_dataset):,}")
print(f"  Effective batch size: {4 * 4} = batch_size × grad_accumulation")

In [ ]:
# ============================================================================
# STEP 6b: Train!
# ============================================================================
# This is where the magic happens.  The Trainer will:
#   1. Iterate over batches of training data
#   2. Compute the causal LM loss (cross-entropy)
#   3. Backpropagate gradients through the LoRA matrices
#   4. Update the LoRA weights with PagedAdamW
#   5. Log loss/learning_rate/GPU memory to the console (and W&B)
#   6. Save checkpoints every N steps
#   7. Run evaluation on the validation set
#
# Expected time: ~30–60 min on T4 (10% subset, 3 epochs)

print("🏋️ Starting training ...")
trainer.train()
print("✅ Training complete!")

In [ ]:
# ============================================================================
# STEP 6c: Save the Adapter and Merged Model
# ============================================================================

# Save JUST the LoRA adapter weights (~50 MB)
# You can reload these later with PeftModel.from_pretrained()
adapter_path = "./results/adapter"
model.save_pretrained(adapter_path)
tokenizer.save_pretrained(adapter_path)
print(f"💾 Adapter saved to {adapter_path}")

# Merge the adapter INTO the base model and save as a standalone model
# This "bakes in" the LoRA weights: W_merged = W_base + (α/r) × B @ A
# The merged model can be loaded without the PEFT library.
merged_model = model.merge_and_unload()
merged_path = "./results/merged_model"
merged_model.save_pretrained(merged_path)
tokenizer.save_pretrained(merged_path)
print(f"💾 Merged model saved to {merged_path}")

## Step 7: Inference — Test Your Model!

Now let's use the fine-tuned model to generate SQL queries from natural language.

In [ ]:
# ============================================================================
# STEP 7: Generate SQL with the Fine-Tuned Model
# ============================================================================

# Inference prompt template — same as training but WITHOUT the answer
INFERENCE_TEMPLATE = """Below is a question about a database along with the schema.
Write the SQL query that answers the question.

### Question:
{question}

### Schema:
{context}

### SQL:
"""


def generate_sql(question: str, schema: str, max_new_tokens: int = 256) -> str:
    """
    Generate a SQL query from a natural-language question and schema.
    
    Args:
        question: The natural language question
        schema: The CREATE TABLE schema(s)
        max_new_tokens: Maximum tokens to generate
    
    Returns:
        The generated SQL query
    """
    prompt = INFERENCE_TEMPLATE.format(question=question, context=schema)
    
    inputs = tokenizer(
        prompt, return_tensors="pt", truncation=True, max_length=512
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.1,          # Low temperature for deterministic SQL
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    generated = outputs[0][inputs["input_ids"].shape[1]:]
    return tokenizer.decode(generated, skip_special_tokens=True).strip()


# ── Try it out! ──────────────────────────────────────────────────────────────
test_question = "How many employees are in each department?"
test_schema = """CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary REAL
)"""

generated_sql = generate_sql(test_question, test_schema)
print(f"Question: {test_question}")
print(f"Generated SQL: {generated_sql}")

In [ ]:
# ============================================================================
# STEP 7b: Test on Examples from the Test Set
# ============================================================================

print("=" * 60)
print("PREDICTIONS ON TEST SET")
print("=" * 60)

for i in range(min(10, len(test_dataset))):
    example = test_dataset[i]
    predicted = generate_sql(example["question"], example["context"])
    
    print(f"\n--- Example {i+1} ---")
    print(f"Question: {example['question']}")
    print(f"Gold SQL: {example['answer']}")
    print(f"Pred SQL: {predicted}")
    print(f"Match:    {'✅' if predicted.strip().lower().rstrip(';') == example['answer'].strip().lower().rstrip(';') else '❌'}")

## Step 8: Evaluation

We evaluate using multiple complementary metrics:

| Metric | What It Measures |
|--------|------------------|
| **Perplexity** | How surprised the model is by the test data (lower = better) |
| **Exact Match** | Does the prediction exactly match the gold SQL? |
| **Execution Accuracy** | Is the generated SQL syntactically valid? |
| **ROUGE-L** | Longest common subsequence overlap (partial credit) |
| **BLEU** | N-gram precision (standard text generation metric) |

In [ ]:
# ============================================================================
# STEP 8: Evaluate on the Test Set
# ============================================================================
import sqlparse
import math

# Limit evaluation examples for speed
MAX_EVAL = 100
eval_subset = test_dataset.select(range(min(MAX_EVAL, len(test_dataset))))

# Generate predictions for all test examples
print(f"Generating predictions for {len(eval_subset)} test examples ...")
predictions = []
for i, example in enumerate(eval_subset):
    pred = generate_sql(example["question"], example["context"])
    predictions.append(pred)
    if (i + 1) % 25 == 0:
        print(f"  [{i+1}/{len(eval_subset)}] done")

gold_answers = eval_subset["answer"]
print(f"✅ Generated {len(predictions)} predictions")

In [ ]:
# ============================================================================
# STEP 8b: Compute All Metrics
# ============================================================================
import evaluate

def normalise_sql(sql):
    """Normalise SQL for fair comparison."""
    return sql.strip().lower().rstrip(";").strip()

# ── Exact Match ──────────────────────────────────────────────────────────────
exact_matches = sum(
    1 for p, g in zip(predictions, gold_answers)
    if normalise_sql(p) == normalise_sql(g)
)
exact_match_acc = exact_matches / len(predictions)

# ── Execution Accuracy (Syntax Check) ────────────────────────────────────────
valid_sql = 0
for sql in predictions:
    try:
        parsed = sqlparse.parse(sql.strip())
        if parsed and str(parsed[0]).strip():
            valid_sql += 1
    except Exception:
        pass
exec_acc = valid_sql / len(predictions)

# ── ROUGE-L ──────────────────────────────────────────────────────────────────
rouge = evaluate.load("rouge")
rouge_results = rouge.compute(
    predictions=predictions, references=gold_answers, rouge_types=["rougeL"]
)

# ── BLEU ─────────────────────────────────────────────────────────────────────
bleu = evaluate.load("bleu")
bleu_results = bleu.compute(
    predictions=predictions, references=[[g] for g in gold_answers]
)

# ── Print Results ────────────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("📊 EVALUATION RESULTS")
print("=" * 50)
print(f"  Exact Match:         {exact_match_acc:.2%} ({exact_matches}/{len(predictions)})")
print(f"  Execution Accuracy:  {exec_acc:.2%} ({valid_sql}/{len(predictions)})")
print(f"  ROUGE-L:             {rouge_results['rougeL']:.4f}")
print(f"  BLEU:                {bleu_results['bleu']:.4f}")
print("=" * 50)

## 🎉 Congratulations!

You've successfully fine-tuned a 7B parameter LLM using QLoRA!

### Next Steps

1. **Scale up**: Set `SUBSET_FRACTION = 1.0` and train on the full dataset
2. **Experiment**: Try different LoRA ranks (8, 32, 64) and see how it affects quality
3. **Deploy**: Use the `inference.py` script to serve your model via vLLM or FastAPI
4. **Track**: Enable Weights & Biases (`report_to="wandb"`) to compare experiments

### Key Takeaways

| What | How |
|------|-----|
| Memory savings | 4-bit quantisation: 14 GB → 4.5 GB |
| Parameter efficiency | LoRA: train 0.4% of parameters |
| Training speed | 3 epochs on 10% data in ~30 min (T4) |
| Serving | vLLM for 2–4× throughput vs HF generate() |